#### 用于根据阈值生成Lesion segmentation

In [4]:
import nibabel as nib
import numpy as np
from scipy.ndimage import label, generate_binary_structure
from typing import Dict

def compute_suvmax_in_mask(pet_nifti_path: str, mask_nifti_path: str) -> float:
    """返回 mask 区域内的 SUVmax；若 mask 为空则抛出 ValueError。"""
    pet_img = nib.load(pet_nifti_path)
    mask_img = nib.load(mask_nifti_path)

    pet_data = pet_img.get_fdata(dtype=np.float32)
    mask_data = mask_img.get_fdata(dtype=np.float32)

    masked_values = pet_data[mask_data > 0]
    if masked_values.size == 0:
        raise ValueError(f"No positive voxels in mask: {mask_nifti_path}")

    return float(masked_values.max())

# 使用示例
pet_path = "/home/yaobo/Project/Lung-Cancer-Subtyping-Classification-V4.0/data/1_preprocessed/sub-AKHABDALLAADELAHMED20091023/sub-AKHABDALLAADELAHMED20091023_pet_desc-suv.nii.gz"
mask_path = "/home/yaobo/Project/Lung-Cancer-Subtyping-Classification-V4.0/data/1_preprocessed/sub-AKHABDALLAADELAHMED20091023/sub-AKHABDALLAADELAHMED20091023_seg-lesion.nii.gz"

suv_max = compute_suvmax_in_mask(pet_path, mask_path)
lesion_threshold = 0.41 * suv_max
print(f"SUVmax: {suv_max:.3f}, threshold: {lesion_threshold:.3f}")

SUVmax: 24.243, threshold: 9.939


In [ ]:
def segment_lesion_with_threshold(pet_path: str, mask_path: str, threshold: float) -> np.ndarray:

    pet_img = nib.load(pet_path)
    mask_img = nib.load(mask_path)
    pet = pet_img.get_fdata(dtype=np.float32)
    mask = mask_img.get_fdata(dtype=np.float32) > 0

    candidates = pet >= threshold
    if not np.any(candidates):
        return np.zeros_like(pet, dtype=np.uint8)

    struct = generate_binary_structure(rank=3, connectivity=3)
    labeled, num = label(candidates, structure=struct)

    sizes = {lbl: int((labeled == lbl).sum()) for lbl in range(1, num + 1)
             if np.any((labeled == lbl) & mask)}
    if not sizes:
        return np.zeros_like(pet, dtype=np.uint8)

    keep_label = max(sizes, key=sizes.get)
    seg = (labeled == keep_label).astype(np.uint8)

    return seg

seg_output_path = '../../outputs/temp/seg/sub-AKHABDALLAADELAHMED20091023_seg-lesion_desc-threshold.nii.gz'

import os
os.makedirs(os.path.dirname(seg_output_path), exist_ok=True)
segmentation = segment_lesion_with_threshold(pet_path, mask_path, lesion_threshold)
nib.save(nib.Nifti1Image(segmentation, nib.load(pet_path).affine, nib.load(pet_path).header), seg_output_path)
print(f"Segmentation voxels: {int(segmentation.sum())}")

Segmentation voxels: 12470
